In [131]:
import pandas as pd
from scipy.stats import truncnorm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [132]:
df = pd.read_excel("../data/processed/preprocessed_final.xlsx")

In [133]:
# Step 1: Select features and target
feature_cols = [
    'acceptance_rate',
    'undergrad_gpa',
    'gre_quantitative_reasoning',
    'gre_verbal_reasoning',
    'analytical_writing'
]


In [134]:
target_col = 'decision_encoded'

In [135]:
# Step 2: Drop rows with missing target
df_model = df.dropna(subset=[target_col])


In [136]:
# Step 3: Extract X and y
X = df_model[feature_cols]
y = df_model[target_col]


In [137]:
# Step 4: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [138]:
# Output the shape of splits
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((68768, 5), (17192, 5), (68768,), (17192,))

In [139]:
# Features to use for regression
feature_cols = [
    'acceptance_rate',
    'undergrad_gpa',
    'gre_quantitative_reasoning',
    'gre_verbal_reasoning',
    'analytical_writing'
]

In [140]:
# Drop rows with missing values in features or target
df_model = df[feature_cols + ['decision_encoded']].dropna()


In [141]:
# Split into features (X) and target (y)
X = df_model[feature_cols]
y = df_model['decision_encoded']


In [142]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [143]:
# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [144]:
# Predict
y_pred = model.predict(X_test)

In [145]:
# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [146]:
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

Mean Squared Error: 0.2001
R² Score: 0.0610


___________________________________________________________________________________

In [147]:

# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Evaluate
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest - Mean Squared Error: {mse_rf:.4f}")
print(f"Random Forest - R² Score: {r2_rf:.4f}")


Random Forest - Mean Squared Error: 0.2104
Random Forest - R² Score: 0.0128


________________________________________________________________________________

In [148]:
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [149]:
y_pred_xgb = xgb_model.predict(X_test)


In [150]:
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

In [151]:
print(f"XGBoost - Mean Squared Error: {mse_xgb:.4f}")
print(f"XGBoost - R² Score: {r2_xgb:.4f}")

XGBoost - Mean Squared Error: 0.1977
XGBoost - R² Score: 0.0724


________________________________________________________________________________

In [152]:
# Define parameter grid to search
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [153]:
# Initialize base model
rf = RandomForestRegressor(random_state=42)


In [154]:
# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)


In [155]:
# Run grid search
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='r2', verbose=1)

In [156]:
# Best model from search
best_rf = grid_search.best_estimator_

In [157]:
y_pred_best = best_rf.predict(X_test)
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

In [158]:
print("Best Parameters:", grid_search.best_params_)
print(f"Best Random Forest - MSE: {mse_best:.4f}, R²: {r2_best:.4f}")

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Random Forest - MSE: 0.1980, R²: 0.0712


In [159]:
df['decision_encoded'].value_counts()


decision_encoded
1.0    37245
0.0    35986
0.5    12729
Name: count, dtype: int64

# Flow

- Make sure data is distributed evenly
- encode target variable
- use one hot encoding for "string" data columns
- run feature engineering
    -  to reduce number of features
- check correlation between columns
- run model training
- run eval on test dataset
- create api endpoint for the model